Data and meta-data available [here](http://download.geonames.org/export/dump/)

In [2]:
import sys,os,re,csv,urllib2
sys.path.append('/usr/local/lib/python2.7/dist-packages/')
import pymysql
import pandas as pd
from secrets import *
import BeautifulSoup as bs4

### Grab files

In [11]:
soup=bs4.BeautifulSoup(urllib2.urlopen('http://download.geonames.org/export/dump/').read())

In [16]:
for link in soup.findAll('a'):
    try:
        if re.search(r'[A-Z]{2,2}'+'\.zip',link.get('href')):
            print(link.get('href'))
            os.system('wget http://download.geonames.org/export/dump/'+link.get('href'))
    except:
        pass
    

In [49]:
data=pd.read_csv('files/AE.txt',sep='\t',header=None)

In [52]:
data.head()

In [51]:
names=[]
names.append('id')
names.append('name')
names.append('ascii_name')
names.append('alternate_names')
names.append('lat')
names.append('long')
names.append('feature') # feature class http://www.geonames.org/export/codes.html
names.append('feature code')
names.append('country')
names.append('country_alt') # weird other country something
names.append('admin1')
names.append('admin2')
names.append('admin3')
names.append('admin4')
names.append('population')
names.append('elevation')
names.append('elevation_dem')
names.append('timezone')
names.append('mod') # Date modified

In [53]:
data.columns=names

In [67]:
data.head(10)

,id,name,ascii_name,alternate_names,lat,long,feature,feature code,country,country_alt,admin1,admin2,admin3,admin4,population,elevation,elevation_dem,timezone,mod
0,100392,Suhūl az̧ Z̧afrah,Suhul az Zafrah,"Suhul az Zafrah,Suhūl az̧ Z̧afrah",22.75000,53.16667,T,PLN,AE,NaN,0,NaN,NaN,NaN,0,NaN,119,Asia/Dubai,2013-07-04
1,101999,Shudūq al Jawwān,Shuduq al Jawwan,"Shuduq al Jawwan,Shudūq al Jawwān",22.83333,53.41667,T,DUNE,AE,NaN,0,NaN,NaN,NaN,0,NaN,96,Asia/Dubai,2013-07-04
2,112658,Lesser Tunb,Lesser Tunb,"Jazireh-Ye-Tonb-e-Kuchak,Jazireh-e Nabiyo Tomb...",26.24194,55.14722,T,ISL,AE,NaN,11,NaN,NaN,NaN,0,NaN,3,Asia/Tehran,2015-02-05
3,112659,Greater Tunb,Greater Tunb,"Greater Tunb,Jazireh-ye Tonb-e Bozorg,Jazīreh-...",26.26389,55.30500,T,ISL,AE,NaN,11,NaN,NaN,NaN,0,NaN,29,Asia/Tehran,2015-02-05
4,251121,Wādī Shīk,Wadi Shik,"Wadi Chich,Wadi Shik,Wādī Chich,Wādī Shīk",24.21667,55.76667,H,WAD,AE,NaN,NaN,NaN,NaN,NaN,0,NaN,290,Asia/Dubai,2013-03-19
5,251154,Wādī Sumaynī,Wadi Sumayni,"Wadi Samain,Wadi Sumayni,Wādī Samain,Wādī Sumaynī",24.83333,55.78333,H,WAD,AE,NaN,0,NaN,NaN,NaN,0,NaN,215,Asia/Dubai,2013-07-06
6,285895,Jabal Zārūb,Jabal Zarub,"Jabal Zarub,Jabal Zārūb,Umm Bar,Umm Barr",24.20778,55.96417,T,MT,AE,NaN,0,NaN,NaN,NaN,0,NaN,553,Asia/Dubai,2012-04-05
7,286019,Ţawī ‘Uwayyir,Tawi `Uwayyir,"Tawi `Uwayyir,Ţawī ‘Uwayyir",24.14583,55.97194,H,WLL,AE,NaN,1,NaN,NaN,NaN,0,NaN,393,Asia/Dubai,2013-08-06
8,286076,Kharmat Umm al ‘Awsh,Kharmat Umm al `Awsh,"Kharimat Umm al Aush,Kharmat Umm al `Awsh,Khar...",23.43333,55.41667,T,TRGD,AE,NaN,0,NaN,NaN,NaN,0,NaN,130,Asia/Dubai,2013-07-06
9,286176,Ţawī al ‘Awqān,Tawi al `Awqan,"Tawi al `Awqan,`Awqan,Ţawī al ‘Awqān,‘Awqān",23.63333,55.55000,L,LCTY,AE,NaN,1,NaN,NaN,NaN,0,NaN,172,Asia/Dubai,2012-04-05


### Grab admin 1 names

In [115]:
!wget http://download.geonames.org/export/dump/admin1CodesASCII.txt

In [8]:
admin1=pd.read_csv('admin1CodesASCII.txt',sep='\t',header=None,names=['code','name','ascii_name','pop'])

In [9]:
admin1.head()

In [10]:
admin1['country']=admin1.code.apply(lambda x:x.partition('.')[0])
admin1['admin_code']=admin1.code.apply(lambda x:x.partition('.')[2])

In [11]:
admin1.head()

### Create Tables

In [12]:
conn = pymysql.connect(host='localhost',user='root',passwd=PW, db='geo',charset='utf8')
cur = conn.cursor()

In [44]:
res=cur.execute('create table admin1 (name varchar(100) character set utf8, ascii_name varchar(100) character set utf8, country varchar(2),admin varchar(10))',)

In [48]:
res=cur.execute('create table places (name varchar(100) character set utf8,clean_name varchar(100) character set utf8, lat float,lon float,country varchar(2),pop int, elevation mediumint)',)

### Add rows for admin 1

In [30]:
cmd="INSERT INTO admin1 (ascii_name,country,admin) VALUES (%s,%s,%s)"

In [19]:
admin1['name'].fillna("",inplace=True)

In [20]:
admin1['admin_code']=admin1['admin_code'].apply(str)

In [28]:
admin1.head()

In [46]:
for row in admin1.iterrows():
    temp=(row[1]['ascii_name'],row[1]['country'],row[1]['admin_code'])
    res=cur.execute(cmd,temp)
    conn.commit()

### Add rows for places

In [24]:
cmd="INSERT INTO places (name,lat,lon,country,pop,elevation) VALUES (%s,%s,%s,%s,%s,%s)"

In [25]:
for row in data.iterrows():
#    print row[0],row[1]
    temp=(row[1]['name'],row[1].lat,row[1]['long'],row[1].country,row[1].population,row[1].elevation_dem)
    print temp
    
    res=cur.execute(cmd,temp)
    conn.commit()
    
    assert cur.rowcount==1

In [93]:
cmd="INSERT INTO places (name,lat,lon,country,pop,elevation) VALUES (%s,%s,%s,%s,%s,%s)"

In [94]:
res=cur.execute(cmd,("test",99.9,99.9,"TE",999,999))

In [ ]:
res=cur.execute()

In [48]:
cmd="INSERT INTO `places` (`name`) VALUES (%s)"

In [95]:
conn.commit()

In [96]:
cur.rowcount